In [ ]:
!unzip "/content/drive/MyDrive/Datasets/mms/spectrograms.zip"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
from PIL import Image

def crop_image(image_path, left, upper, right, lower, output_path):
    with Image.open(image_path) as img:
        cropped_img = img.crop((left, upper, right, lower))
        cropped_img.save(output_path)


In [ ]:
from PIL import Image

def threshold_image(image_path):
    img = Image.open(image_path)
    
    # img = img.convert('L')
    
    img = img.point(lambda p: 255 if p > 165 else p)
    img = img.point(lambda p: 110 if p < 130  else p)
    
    # Save the modified image
    img.save("thresholded_image.jpg")


In [ ]:
def pre_process_img(source_filepath, dest_filepath):

  #crop the image
  left, upper, right, lower = 80, 60, 475, 425
  with Image.open(source_filepath) as img:
      cropped_img = img.crop((left, upper, right, lower))
  
  #make it greyscale
  grey_img = cropped_img.convert('L')

  #pixel transform
  grey_img = grey_img.point(lambda p: 255 if p > 165 else p)
  grey_img = grey_img.point(lambda p: 110 if p < 130  else p)
  print(np.asarray(grey_img))
  #save image
  # grey_img.save(dest_filepath)
  # print(np.asarray(grey_img).shape)
  return np.asarray(grey_img)


In [ ]:

# pre_process_img('spectrograms/sawing/file1.png', 'file1.png')

In [ ]:
# pre_process_img('spectrograms/ambient/file1.png', 'spectrograms/transformed/ambient/file1.png')

In [ ]:
# import multiprocessing as mp
# import os

# num_processes = mp.cpu_count()
# pool = mp.Pool(num_processes)

# out_dir = "spectrograms/transformed/ambient/"
# in_dir = "spectrograms/ambient/"

# if __name__ == '__main__':

#     if not os.path.exists(out_dir):
#         os.makedirs(out_dir)

#     files = os.listdir(in_dir)

#     # Create a pool of processes
#     num_processes = mp.cpu_count()
#     pool = mp.Pool(num_processes)

#     # Use the pool to process the files
#     results = []
#     for f in files:
#         result = pool.apply_async(pre_process_img, args=(in_dir + f, out_dir + f))
#         results.append(result)

#     # Wait for all processes to complete
#     for result in results:
#         result.wait()

#     # Close the pool to free up resources
#     pool.close()
#     pool.join()

In [ ]:
# !zip -r "transformed.zip" "spectrograms/transformed/"

In [ ]:
#cnn with 2 layers

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.callbacks import EarlyStopping

# Define the input shape
# Define the input shape
input_shape = (365, 395, 1)

# Define the model
model = Sequential()

# Add a convolutional layer with 16 filters and a 3x3 kernel size
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Flatten the output of the convolutional layer
model.add(Flatten())

# Add the output layer with 2 neurons for the 2 classes (sawing and not sawing)
model.add(Dense(2, activation='softmax'))

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()


# Define the data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the training data from the directory
train_data = data_generator.flow_from_directory(
    'spectrograms/transformed/',
    target_size=(365, 395),
    batch_size=256,
    color_mode='grayscale',
    class_mode='categorical'
)

early_stop = EarlyStopping(monitor='loss', patience=3)

# Train the model for 10 epochs
model.fit(train_data, epochs=3, callbacks=[early_stop])



In [ ]:
model.save('cnn_model_v1.h5')

In [ ]:
from keras.models import load_model
from PIL import Image
import numpy as np
from tensorflow.keras.utils import load_img
import tensorflow as tf

# Load the saved model from a file
loaded_model = load_model('cnn_model_v1.h5')

pre_process_img("spectrograms/sawing/file112.png", "test7.png")


# Load the image
img = tf.keras.utils.load_img('test7.png', target_size=(365, 395))


# Convert the image to a numpy array
# x = image.img_to_array(img)
x = np.asarray(img)
print(x)
print('*'*10)

# Preprocess the image using the same method as the training data
x = x / 255.

# Reshape the input data to match the expected shape of the model
x = np.expand_dims(x, axis=0)
# print("x", x.shape)

# Make the prediction using the model
preds = model.predict(x)

# Print the predicted class
print(preds.argmax())

if preds.argmax() == 1:
  print('sawing!!')
else:
  print('not sawing!')


In [ ]:
def pre_process_img2(source_filepath):

  #crop the image
  left, upper, right, lower = 80, 60, 475, 425
  with Image.open(source_filepath) as img:
      cropped_img = img.crop((left, upper, right, lower))
  
  #make it greyscale
  grey_img = cropped_img.convert('L')

  #pixel transform
  grey_img = grey_img.point(lambda p: 255 if p > 165 else p)
  grey_img = grey_img.point(lambda p: 110 if p < 130  else p)

  return np.asarray(grey_img)


In [ ]:
#libraries
from keras.models import load_model
from PIL import Image
import numpy as np
# from tensorflow.keras.utils import load_img
import tensorflow as tf


def predict_spectrogram(filepath):
  model = load_model('/content/drive/MyDrive/Datasets/mms/cnn_model_v1.hdf5')
  img_data = pre_process_img2(filepath)
  x = img_data / 255.

  x = np.expand_dims(x, axis=0)

  preds = model.predict(x)
  return True if preds.argmax()==1 else False
predict_spectrogram('/content/drive/MyDrive/Datasets/spectogramImgs/file1.png')

In [ ]:
!pip show tensorflow
!pip show keras

In [ ]:
!pip show flatbuffers tensorboard 

In [ ]:
!pip show protobuf

In [ ]:
!pip show Pillow

In [ ]:
!pip show numpy

In [ ]:
!unzip "/content/drive/MyDrive/Datasets/mms/spectrograms.zip"

In [ ]:
import os
os.makedirs('spectrograms/transformed-v3/sawing/')

In [ ]:
import os
os.makedirs('spectrograms/transformed-v3/ambient/')

*************************************


In [ ]:
from PIL import Image
import numpy as np

def pre_process_img_v3(source_filepath,f):

    # crop the image
    left, upper, right, lower = 170, 200, 350, 380
    with Image.open(source_filepath) as img:
        cropped_img = img.crop((left, upper, right, lower))

    # make it greyscale
    grey_img = cropped_img.convert('L')

    # pixel transform
    grey_img = grey_img.point(lambda p: 255 if p > 165 else p)
    grey_img = grey_img.point(lambda p: 110 if p < 130 else p)

    # convert to numpy array 
    np_img = np.asarray(grey_img)

    # Image.fromarray(np_img.save('random-test.png'))
    # img = Image.fromarray(np_img)
    # img.save(f)

    return np_img
# pre_process_img_v3('/content/spectrograms/sawing/file1090.png', 'random112.png')

In [ ]:
print(len(os.listdir('spectrograms/transformed-v3/ambient/')))

In [ ]:

import multiprocessing as mp
import os

num_processes = mp.cpu_count()
pool = mp.Pool(num_processes)

out_dir = "spectrograms/transformed-v3/sawing/"
in_dir = "spectrograms/sawing/"

if __name__ == '__main__':

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    files = os.listdir(in_dir)

    # Create a pool of processes
    num_processes = mp.cpu_count()
    pool = mp.Pool(num_processes)

    # Use the pool to process the files
    results = []
    for f in files:
        result = pool.apply_async(pre_process_img_v3, args=(in_dir + f, out_dir + f))
        results.append(result)

    # Wait for all processes to complete
    for result in results:
        result.wait()

    # Close the pool to free up resources
    pool.close()
    pool.join()

In [ ]:
print(len(os.listdir('spectrograms/transformed-v3/sawing/')))

In [ ]:
import multiprocessing as mp
import os

num_processes = mp.cpu_count()
pool = mp.Pool(num_processes)

out_dir = "spectrograms/transformed-v3/ambient/"
in_dir = "spectrograms/ambient/"

if __name__ == '__main__':

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    files = os.listdir(in_dir)

    # Create a pool of processes
    num_processes = mp.cpu_count()
    pool = mp.Pool(num_processes)

    # Use the pool to process the files
    results = []
    for f in files:
        result = pool.apply_async(pre_process_img_v3, args=(in_dir + f, out_dir + f))
        results.append(result)

    # Wait for all processes to complete
    for result in results:
        result.wait()

    # Close the pool to free up resources
    pool.close()
    pool.join()

In [ ]:
print(len(os.listdir('/content/spectrograms/transformed-v3/ambient')))

In [ ]:
# cnn model v3 even lightweight

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adagrad

# Define the input shape
input_shape = (180, 180, 1)

# Define the model
model = tf.keras.Sequential([
    Conv2D(4, kernel_size=(2, 2), activation=Activation('swish'), input_shape=input_shape),
    Flatten(),
    Dense(2, activation='sigmoid')
])

# Compile the model with binary cross-entropy loss and Adagrad optimizer
model.compile(loss='binary_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])

# Define the data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the training data from the directory
train_data = data_generator.flow_from_directory(
    'spectrograms/transformed-v3/',
    target_size=(180,180),
    batch_size=256,
    color_mode='grayscale',
    class_mode='categorical'
)

# print(train_data)

# Train the model for 3 epochs
model.fit(train_data, epochs=3)

# Convert the model to TensorFlow Lite format with float16 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
quantized_model = converter.convert()

# Save the quantized model
with open('cnn_model_v2.tflite', 'wb') as f:
    f.write(quantized_model)


In [ ]:
import tensorflow as tf
import numpy as np
# from PIL import Image

def predict_with_tflite(image_path, model_path):

    image = pre_process_img_v3(image_path,'')

    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    image = np.expand_dims(image, axis=-1)
    image = image.astype(np.float32)

    # print(image.shape)

    # Load the model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get the input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make the prediction
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])

    # Return the prediction
    return prediction[0]


prediction = predict_with_tflite('spectrograms/sawing/file7810.png', 'cnn_model_v2.tflite')
prediction_dic = {0:'ambient',1:'sawing'}
print(prediction_dic[np.argmax(prediction)])